<a href="https://colab.research.google.com/github/bsong75/brendensong.github.io/blob/main/9.a.Neural_LR_Insurance_cost%20estimate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.utils.colab.set_colab_file_id('1PiHR7yXPE1FnrF4qWvdfLb5I62AZCS2M')

     |████████████████████████████████| 71kB 6.4MB/s 


# Insurance cost prediction using linear regression

Make a submisson here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from [Kaggle](https://www.kaggle.com/mirichoi0218/insurance).


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lessons. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ai/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ai/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ai/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [ ]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

In [2]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [3]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [4]:
DATASET_URL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [ ]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.describe()

We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [8]:
your_name = "Brenden" # at least 5 characters

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [9]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [13]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [15]:
dataframe.shape

(1271, 7)

Let us answer some basic questions about the dataset. 


**Q: How many rows does the dataset have?**

In [18]:
num_rows = 1271
print(num_rows)

1271


**Q: How many columns doe the dataset have**

In [19]:
num_cols = 7
print(num_cols)

7


**Q: What are the column titles of the input variables?**

In [23]:
dataframe.head()

,age,sex,bmi,children,smoker,region,charges
397,21,male,35.3628,0,no,southeast,16752.362687
211,40,male,35.1975,4,no,northwest,8244.343413
304,57,female,43.3200,2,no,southwest,12772.669070
349,19,male,31.7319,0,no,northwest,1652.090987
733,48,female,31.0821,1,no,northeast,9541.722853


In [239]:
dataframe_temp = dataframe.drop(["charges"], axis=1) 



In [240]:
input_cols = dataframe_temp.columns
input_cols


Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region'], dtype='object')

**Q: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [241]:
categorical_cols = ['sex', 'smoker', 'region']

**Q: What are the column titles of output/target variable(s)?**

In [243]:
output_cols = ["charges"]

In [244]:
print(dataframe.charges.max())
print(dataframe.charges.min())
print(dataframe.charges.mean())
dataframe.charges.describe()
#, dataframe.charges.max(). dataframe.charges.average())
# import numpy as np
# max_val = np.max(output_cols)
# min_val = np.min(output_cols)
# average= np.ave(output_cols)

64408.1322901
1133.092639
13382.321537417465


count     1271.000000
mean     13382.321537
std      12243.449474
min       1133.092639
25%       4776.292399
50%       9397.748919
75%      16747.959991
max      64408.132290
Name: charges, dtype: float64

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ml/aakashns/dataviz-cheatsheet

In [ ]:
# minimum: 1133.09   maximum: 64408   average: 13382

Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [245]:
!pip install jovian --upgrade -q

In [246]:
import jovian

In [247]:
jovian.commit(project='my_project')

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/bsong75/my-project


'https://jovian.ai/bsong75/my-project'

## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [248]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [249]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[21.    ,  1.    , 35.3628,  0.    ,  0.    ,  2.    ],
        [40.    ,  1.    , 35.1975,  4.    ,  0.    ,  1.    ],
        [57.    ,  0.    , 43.32  ,  2.    ,  0.    ,  3.    ],
        ...,
        [19.    ,  0.    , 29.3493,  1.    ,  0.    ,  1.    ],
        [62.    ,  0.    , 28.5   ,  0.    ,  0.    ,  3.    ],
        [46.    ,  1.    , 50.0346,  3.    ,  0.    ,  2.    ]]),
 array([[16752.3626871],
        [ 8244.3434125],
        [12772.66907  ],
        ...,
        [ 2737.9368355],
        [13585.63322  ],
        [ 9033.556251 ]]))

**Q: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [250]:
inputs = torch.tensor(inputs_array, dtype=torch.float32)
targets = torch.tensor(targets_array, dtype=torch.float32)
inputs[0:1]

tensor([[21.0000,  1.0000, 35.3628,  0.0000,  0.0000,  2.0000]])

In [251]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [252]:
dataset = TensorDataset(inputs, targets)

**Q: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [256]:
val_percent = 0.15 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split (dataset, [train_size, val_size]) # Use the random_split function to split dataset into 2 parts of the desired length

Finally, we can create data loaders for training & validation.

**Q: Pick a batch size for the data loader.**

In [257]:
batch_size = 50

In [258]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [259]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[42.0000,  1.0000, 32.2734,  3.0000,  1.0000,  1.0000],
        [23.0000,  0.0000, 27.6165,  2.0000,  0.0000,  0.0000],
        [32.0000,  0.0000, 33.7326,  1.0000,  0.0000,  2.0000],
        [35.0000,  0.0000, 31.9485,  0.0000,  1.0000,  1.0000],
        [39.0000,  0.0000, 47.6520,  0.0000,  0.0000,  2.0000],
        [60.0000,  0.0000, 29.4576,  0.0000,  0.0000,  1.0000],
        [30.0000,  0.0000, 35.2260,  3.0000,  0.0000,  3.0000],
        [51.0000,  1.0000, 42.1800,  0.0000,  0.0000,  3.0000],
        [64.0000,  1.0000, 29.1840,  2.0000,  0.0000,  3.0000],
        [62.0000,  0.0000, 37.5801,  3.0000,  0.0000,  1.0000],
        [46.0000,  1.0000, 22.6347,  0.0000,  0.0000,  1.0000],
        [31.0000,  0.0000, 35.4141,  0.0000,  0.0000,  0.0000],
        [58.0000,  0.0000, 37.5801,  0.0000,  0.0000,  0.0000],
        [31.0000,  1.0000, 32.5983,  1.0000,  0.0000,  1.0000],
        [47.0000,  0.0000, 38.0133,  0.0000,  0.0000,  0.0000],
        [42.0000,  1.0000, 34.20

Let's save our work by committing to Jovian.

In [150]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Committed successfully! https://jovian.ai/bsong75/02-insurance-linear-regression


'https://jovian.ai/bsong75/02-insurance-linear-regression'

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [260]:
input_size = len(input_cols)
output_size = len(output_cols)
print(input_size, output_size)

6 1


In [261]:
import torch.nn.functional as F

**Q: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [333]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)                  # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                          # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.mse_loss(out, targets)                          # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.mse_loss(out,targets)                          # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [334]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [335]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.1047, -0.0213, -0.3862,  0.3312,  0.0623,  0.2677]],
        requires_grad=True), Parameter containing:
 tensor([0.3019], requires_grad=True)]

One final commit before we train the model.

In [318]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/bsong75/02-insurance-linear-regression


'https://jovian.ai/bsong75/02-insurance-linear-regression'

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [336]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [337]:
result = evaluate(model, val_loader) # Use the the evaluate function
print(result)

{'val_loss': 330586336.0}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [338]:
epochs = 100
lr = 1e-50000
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 330586336.0000
Epoch [40], val_loss: 330586336.0000
Epoch [60], val_loss: 330586336.0000
Epoch [80], val_loss: 330586336.0000
Epoch [100], val_loss: 330586336.0000


In [339]:
epochs = 100
lr = 1e-500000
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 330586336.0000
Epoch [40], val_loss: 330586336.0000
Epoch [60], val_loss: 330586336.0000
Epoch [80], val_loss: 330586336.0000
Epoch [100], val_loss: 330586336.0000


In [340]:
epochs = 100
lr = 1e-4
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 142152736.0000
Epoch [40], val_loss: 141338496.0000
Epoch [60], val_loss: 139822688.0000
Epoch [80], val_loss: 131832696.0000
Epoch [100], val_loss: 130413088.0000


In [341]:
epochs = 100
lr = 1e-5
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 129162088.0000
Epoch [40], val_loss: 128950848.0000
Epoch [60], val_loss: 128725984.0000
Epoch [80], val_loss: 128574184.0000
Epoch [100], val_loss: 128241712.0000


In [342]:
epochs = 1000
lr = 1e-6
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 128242728.0000
Epoch [40], val_loss: 128221544.0000
Epoch [60], val_loss: 128195744.0000
Epoch [80], val_loss: 128175184.0000
Epoch [100], val_loss: 128149872.0000
Epoch [120], val_loss: 128121728.0000
Epoch [140], val_loss: 128106696.0000
Epoch [160], val_loss: 128076208.0000
Epoch [180], val_loss: 128060656.0000
Epoch [200], val_loss: 128031688.0000
Epoch [220], val_loss: 128011288.0000
Epoch [240], val_loss: 127983264.0000
Epoch [260], val_loss: 127963016.0000
Epoch [280], val_loss: 127936696.0000
Epoch [300], val_loss: 127919808.0000
Epoch [320], val_loss: 127899832.0000
Epoch [340], val_loss: 127869984.0000
Epoch [360], val_loss: 127843408.0000
Epoch [380], val_loss: 127825088.0000
Epoch [400], val_loss: 127795712.0000
Epoch [420], val_loss: 127774664.0000
Epoch [440], val_loss: 127749632.0000
Epoch [460], val_loss: 127728944.0000
Epoch [480], val_loss: 127705896.0000
Epoch [500], val_loss: 127682040.0000
Epoch [520], val_loss: 127659312.0000
Epoch [540], val

**Q: What is the final validation loss of your model?**

In [343]:
val_loss = 127105360

Let's log the final validation loss to Jovian and commit the notebook

In [345]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Metrics logged.


In [346]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/bsong75/02-insurance-linear-regression


'https://jovian.ai/bsong75/02-insurance-linear-regression'

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q: Complete the following function definition to make predictions on a single input**

In [347]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions =  model(input)               # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [348]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([19.0000,  1.0000, 34.4850,  0.0000,  1.0000,  2.0000])
Target: tensor([32873.8242])
Prediction: tensor(10631.0127)


In [349]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([53.0000,  1.0000, 27.7248,  0.0000,  0.0000,  1.0000])
Target: tensor([9962.1064])
Prediction: tensor(14822.4131)


In [350]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([20.0000,  1.0000, 33.8979,  0.0000,  0.0000,  1.0000])
Target: tensor([1787.2269])
Prediction: tensor(8958.5527)


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to replicate this notebook for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patterns in machine learning from problem-specific details.You can use one of these starer notebooks (just change the dataset):

- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog. 

Don't forget to share your work on the forum: https://jovian.ai/forum/t/linear-regression-and-logistic-regression-notebooks-and-blog-posts/14039

In [351]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/bsong75/02-insurance-linear-regression
[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/bsong75/02-insurance-linear-regression


'https://jovian.ai/bsong75/02-insurance-linear-regression'